In [13]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as mt

import src.crs as crs

# 1. Reco Process

In [107]:
# 1. load_excel
data_path = "data/apt_1.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

# 2. data-preprocessing
p, m = crs.utils.data_preprocessing(xlsx)

# 3. bill calc
min_per = 10
max_per = 80

bc_result = crs.utils.bill_calc(m, p, min_per, max_per)

# 4. normal analysis
n_result = crs.utils.normal_analysis(bc_result)

# 5. mean analysis
m_result = crs.utils.mean_analysis(m, p, min_per, max_per)

# 6. similarity analysis
s_result = crs.utils.similarity_analysis(m, p, min_per, max_per)

/Users/formegusto/Desktop/idoformegusto/development/research/distribution-by-contribution/src/crs/utils/data_preprocessing.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  datas_df[household_name] = household_df[col][3:].to_list()
/Users/formegusto/Desktop/idoformegusto/development/research/distribution-by-contribution/src/crs/utils/data_preprocessing.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  month_usage_df[name] = [


가구 수: 108 / 평균: 288 / 표준편차: 120 / 변동계수: 42
가구 수: 108 / 평균: 260 / 표준편차: 97 / 변동계수: 37
가구 수: 108 / 평균: 273 / 표준편차: 92 / 변동계수: 34
가구 수: 108 / 평균: 254 / 표준편차: 88 / 변동계수: 35
가구 수: 108 / 평균: 240 / 표준편차: 89 / 변동계수: 37
가구 수: 108 / 평균: 233 / 표준편차: 89 / 변동계수: 38
가구 수: 108 / 평균: 326 / 표준편차: 135 / 변동계수: 41
가구 수: 108 / 평균: 362 / 표준편차: 153 / 변동계수: 42
가구 수: 108 / 평균: 231 / 표준편차: 92 / 변동계수: 40
가구 수: 108 / 평균: 238 / 표준편차: 96 / 변동계수: 40
가구 수: 108 / 평균: 251 / 표준편차: 106 / 변동계수: 42
가구 수: 108 / 평균: 288 / 표준편차: 111 / 변동계수: 39
가구 수: 108 / 평균: 270 / 표준편차: 96 / 변동계수: 36
가구 수: 108 / 평균: 245 / 표준편차: 85 / 변동계수: 35


In [127]:
print(bc_result['mgmt_list']['single'][0][20].apart.kwh)

basic = bc_result['mgmt_list']['single'][0][20].apart.basic * 108
elec_rate = bc_result['mgmt_list']['single'][0][20].apart.elec_rate * 108
env = bc_result['mgmt_list']['single'][0][20].apart.env * 108
fuel = bc_result['mgmt_list']['single'][0][20].apart.fuel * 108
elec_bill = basic + elec_rate + env - fuel


print(basic, elec_rate, env, fuel, elec_bill)
print("vat: {}".format(round(elec_bill * 0.1)))
print("fund: {}".format(mt.floor(elec_bill * 0.037 * 0.1) * 10))
print("bill: {}".format(bc_result['mgmt_list']['single'][0][20].bill))

411
654480 4907088 235224 133164 5663628
vat: 566363
fund: 209550
bill: 6439540


In [166]:
households = bc_result['mgmt_list']['single'][0][20].households

In [180]:
print("기본요금",np.array([_.basic for _ in households]).sum())
print("전력량요금",np.array([_.elec_rate for _ in households]).sum())
print("환경",np.array([_.env for _ in households]).sum())
print("연료비",np.array([_.fuel for _ in households]).sum())
print("vat",np.array([_.vat for _ in households]).sum())
print("fund",np.array([_.fund for _ in households]).sum())
print("fund",np.array([_.guarantee for _ in households]).sum())
print("fund",np.array([_.elec_bill_vat_fund for _ in households]).sum())

기본요금 197480
전력량요금 3118820
환경 164805
연료비 93312
vat 334205
fund 123150
fund 50000
fund 3798980


In [162]:
print(fuel - np.array([_.fuel for _ in bc_result['mgmt_list']['single'][0][20].households]).sum())

39852


In [163]:
print(basic - np.array([_.basic for _ in bc_result['mgmt_list']['single'][0][20].households]).sum())
print(np.array([_.fund for _ in bc_result['mgmt_list']['single'][0][20].households]).sum())

457000
123150


In [157]:
print(round(elec_bill * 0.1) - np.array([_.vat for _ in bc_result['mgmt_list']['single'][0][20].households]).sum())
print((mt.floor(elec_bill * 0.037 * 0.1) * 10) - np.array([_.fund for _ in bc_result['mgmt_list']['single'][0][20].households]).sum())

232158
86400


In [135]:
fuel * 0.3

39949.2

In [131]:
bc_result['bill']['single']["30"][1]

6439540.0

In [138]:
123 * 5.3 * 108

70405.2

In [132]:
# 사용량 파싱
percentage = 30

households_cnt = len(m.iloc[0][1:])
households_kwh = m.iloc[0][1:].sum()

APT = round((households_kwh * 100) / (100 - percentage))
public_kwh = APT - households_kwh

APT_bill = bc_result['bill']['single']["30"][1]
public_bill = bc_result['public_bill']['single']["30"][1]
households_bill = APT_bill - public_bill

print("아파트 전체 사용량 : {}kwh, {}원"\
      .format(APT, int(APT_bill)))
print("세대부 사용량 : {}kwh, {}원, 세대부 가구 수 : {}가구"\
      .format(households_kwh, int(households_bill),households_cnt))
print("공용부 사용량 : {}kwh, {}원, 공용부 PERCENTAGE : {}%"\
      .format(public_kwh, int(public_bill),round(public_kwh / APT * 100)))

아파트 전체 사용량 : 44434kwh, 6439540원
세대부 사용량 : 31104kwh, 3798980원, 세대부 가구 수 : 108가구
공용부 사용량 : 13330kwh, 2640560원, 공용부 PERCENTAGE : 30%


In [88]:
bill_rate = round(APT / households_cnt)
print("요금 산정 사용량 : {}kwh".format(int(bill_rate)))

요금 산정 사용량 : 411kwh
